Computing stats for single cell RNA bubbleplot

In [3]:
import scanpy as sc
import numpy as np
import pandas as pd
import anndata as ad
import synapseclient

In [4]:
syn = synapseclient.Synapse()
syn.login()

Welcome, heimann!



In [5]:
#loading h5ad file after qc, described in 'msk_data_prep.ipynb'
entity = syn.get('syn52118064')
adata = sc.read_h5ad(entity.path)
adata

AnnData object with n_obs × n_vars = 16475 × 25090
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'mito_frac', 'RBP_frac', 'batch', 'donor_id', 'treatment', 'procedure', 'author_cell_type', 'cell_type_broad', 'clusters', 'treatment_categorized', 'subtype', 'H_treatment', 'H_subtype', 'cell_type_ontology_term_id', 'tissue_ontology_term_id', 'disease_ontology_term_id', 'organism_ontology_term_id', 'is_primary_data', 'development_stage_ontology_term_id', 'assay_ontology_term_id', 'sex_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'suspension_type', 'HTAN_Biospecimen_ID', 'HTAN_Participant_ID', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage'
    var: 'ensembl_gene_id', 'feature_is_filtered', 'feature_reference', 'feature_biotype'
    uns: 'log1p', 'neighbors', 'schema_version', 'title'
    obsm: 'X_pca', 'X_umap'
    layers: 'counts', 'log2(X+0.1)', 'normalized'
   

In [8]:
#we'll need to use the R package that queries the iAtlas database, the libraries below are necessary to call R code

#getting the list of immunomodulators from CRI iAtlas
from rpy2.robjects.packages import importr
import rpy2.robjects as ro

iatlasGraphQLClient = importr('iatlasGraphQLClient')
immunomodulators = ro.conversion.rpy2py(iatlasGraphQLClient.query_immunomodulators())
immunomodulators

entrez,hgnc,friendly_name,...,immune_checkpoint,super_category,publications
135,'ADORA2A','ADORA2A',...,'Inhibito...,'Receptor',ListSexpV...
383,'ARG1','ARG1',,'Inhibito...,'Other',ListSexpV...
151888,'BTLA','BTLA',,'Inhibito...,'Receptor',ListSexpV...
11119,'BTN3A1','BTN3A1',,'Stimulat...,'Co-inhib...,ListSexpV...
...,...,...,,...,...,...
7422,'VEGFA','VEGFA',,'Inhibito...,'Ligand',ListSexpV...
7423,'VEGFB','VEGFB',,'Inhibito...,'Ligand',ListSexpV...
64115,'VSIR','VISTA',,'Inhibito...,'Co-inhib...,ListSexpV...
79679,'VTCN1','VTCN1',,'Inhibito...,'Co-inhib...,ListSexpV...


In [9]:
#get a list of the genes that are present in the dataset & are immunomodulators
genes = pd.Series(adata.var[np.isin(adata.var.index, immunomodulators[2])].index)
genes

0      ADORA2A
1         ARG1
2         BTLA
3       BTN3A1
4       BTN3A2
        ...   
57         TNF
58    TNFRSF18
59       VEGFA
60       VEGFB
61       VTCN1
Name: feature_name, Length: 62, dtype: category
Categories (25090, object): ['A1BG', 'A1CF', 'A2M', 'A2ML1', ..., 'ZYG11B', 'ZYX', 'ZZEF1', 'ZZZ3']

In [12]:
#compute how many counts map to each cell type
freq_cell_types = adata.obs["cell_type_broad"].value_counts()
freq_cell_types

cell_type_broad
T cells    11697
Myeloid     2994
NK          1104
B cell       680
Name: count, dtype: int64

In [13]:
#create grid with all cell x gene combinations
lp1, lp2 = pd.core.reshape.util.cartesian_product([freq_cell_types.index, genes])
cell_gene = pd.DataFrame(dict(cell=lp1, gene=lp2))
cell_gene

,cell,gene
0,T cells,ADORA2A
1,T cells,ARG1
2,T cells,BTLA
3,T cells,BTN3A1
4,T cells,BTN3A2
5,T cells,CD27
6,T cells,CD276
7,T cells,CD28
8,T cells,CD40
9,T cells,CD40LG


We need to compute:
- % of cell of a given type that expresses a gene (count expr by type/freq of cell type)
- mean expr value

In [14]:
def get_expr_by_cell(cell_type, gene):
    expr = adata[adata.obs.cell_type_broad == cell_type].to_df(layer="counts")[gene]
    counts = sum(expr != 0)
    if(counts>0): avg = (expr[expr != 0]).mean()
    else: avg = 0 #technically this is wrong, but plotting libraries crash with NAs or characters
    return counts, avg

In [22]:
# Function to apply to each row
def apply_function(row):
    counts, avg = get_expr_by_cell(row['cell'], row['gene'])  
    return pd.Series({'counts': counts, 'avg': avg})

# Apply the function to each row and concatenate the results
result = pd.concat([cell_gene, cell_gene.apply(apply_function, axis=1)], axis=1)


In [23]:
result.iloc[0:10]

,cell,gene,counts,avg
0,T cells,ADORA2A,2.0,1.902162
1,T cells,ARG1,4.0,1.763940
2,T cells,BTLA,474.0,1.909067
3,T cells,BTN3A1,2802.0,2.027277
4,T cells,BTN3A2,5087.0,2.110082
5,T cells,CD27,4794.0,2.162383
6,T cells,CD276,51.0,1.879961
7,T cells,CD28,2263.0,2.028319
8,T cells,CD40,148.0,1.960982
9,T cells,CD40LG,1481.0,2.111081


In [25]:
#Now we compute the % of cells from a given cell type that have expression for a gene
result = pd.merge(result, freq_cell_types, left_on='cell', right_index=True)
result['perc_expr'] = result['counts'] / result['count']
result.iloc[0:10]

,cell,gene,counts,avg,count,perc_expr
0,T cells,ADORA2A,2.0,1.902162,11697,0.000171
1,T cells,ARG1,4.0,1.763940,11697,0.000342
2,T cells,BTLA,474.0,1.909067,11697,0.040523
3,T cells,BTN3A1,2802.0,2.027277,11697,0.239549
4,T cells,BTN3A2,5087.0,2.110082,11697,0.434898
5,T cells,CD27,4794.0,2.162383,11697,0.409849
6,T cells,CD276,51.0,1.879961,11697,0.004360
7,T cells,CD28,2263.0,2.028319,11697,0.193468
8,T cells,CD40,148.0,1.960982,11697,0.012653
9,T cells,CD40LG,1481.0,2.111081,11697,0.126614


In [26]:
#change column names
result.columns = ["cell", "gene", "counts", "avg", "Freq", "perc_expr"]
result.iloc[0:10]

,cell,gene,counts,avg,Freq,perc_expr
0,T cells,ADORA2A,2.0,1.902162,11697,0.000171
1,T cells,ARG1,4.0,1.763940,11697,0.000342
2,T cells,BTLA,474.0,1.909067,11697,0.040523
3,T cells,BTN3A1,2802.0,2.027277,11697,0.239549
4,T cells,BTN3A2,5087.0,2.110082,11697,0.434898
5,T cells,CD27,4794.0,2.162383,11697,0.409849
6,T cells,CD276,51.0,1.879961,11697,0.004360
7,T cells,CD28,2263.0,2.028319,11697,0.193468
8,T cells,CD40,148.0,1.960982,11697,0.012653
9,T cells,CD40LG,1481.0,2.111081,11697,0.126614


In [27]:
#The MSK data has different names for cells than what we will use in iAtlas, so let's update them
result['cell'] = result['cell'].replace(['T cells', 'Myeloid'], ['T cell', 'myeloid cell'])

[WARNING] /var/folders/dv/r6k_tzv51_965ns1n7_w6_hh0000gn/T/ipykernel_9178/3464894311.py:2: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  result['cell'] = result['cell'].replace(['T cells', 'Myeloid'], ['T cell', 'myeloid cell'])



In [29]:
#Add dataset info and save data into file
result['dataset'] = "MSK"
result.to_csv('msk_bubble_plot_df.tsv', sep='\t', index=False)
file_entity = synapseclient.File('msk_bubble_plot_df.tsv', 'syn61486284')
file_entity = syn.store(file_entity)

Uploading to Synapse storage: 100%|██████████| 15.7k/15.7k [00:00<00:00, 26.4kB/s, msk_bubble_plot_df.tsv]
